So, I've moved the data saved on my machine (I'm calling these datasets "historic data") into a MongoDB database. The historic data covers from about April 2020 to October 2020. Before I start adding more recent data to the database, I'm going to build a basic class to represent all tradeable items. The idea is to instantiate this `Tradeable` class for any item I want and have the most recent data (last 180 days) available as a pandas.DataFrame. 

I'll populate the DataFrame using data from the Official Old School Runescape Grand Exchange API. But, there are 2 known issues with this approach. The first issue is that volume data (the number of units sold on a date) is not available through API requests. Instead, I'll scrape the HTML of the item page and progromatically collect volume data. This solution comes with its own set of small nuances that I'll explore more below.

The next issue is that, unlike every other item, the "Old School Bond" does not have volume data available on its OSRS GE webpage. Old School Bonds can be bought with in-game gold as well as real money. So, I assume Jagex intentionally keeps this data hidden from the public. However, the (closing) price and average price for Old School Bonds are available and nice to have, so I'll have to account for this case when building the class.

# Making Requests to OSRS GE API #

In [1]:
import requests
import json
import pandas as pd
from osrsbox import items_api

## Formatting API Endpoints ##
I'm going to need the item name and unique ID to format API endpoints. Let's start by grabbing the unique ID and names of items on the Grand Exchange.

In [2]:
#iter object of all items in the osrsbox-db
all_available_items = items_api.load().__iter__()

In [3]:
#filter for items available on the Grand Exchange
#then map the unique ID to the item name
tradeables_ids_to_names = {item.id:item.name.lower() for item in list(all_available_items) if item.tradeable_on_ge==True}
tradeables_ids_to_names

{2: 'cannonball',
 6: 'cannon base',
 8: 'cannon stand',
 10: 'cannon barrels',
 12: 'cannon furnace',
 28: 'insect repellent',
 30: 'bucket of wax',
 36: 'candle',
 39: 'bronze arrowtips',
 40: 'iron arrowtips',
 41: 'steel arrowtips',
 42: 'mithril arrowtips',
 43: 'adamant arrowtips',
 44: 'rune arrowtips',
 45: 'opal bolt tips',
 46: 'pearl bolt tips',
 47: 'barb bolttips',
 48: 'longbow (u)',
 50: 'shortbow (u)',
 52: 'arrow shaft',
 53: 'headless arrow',
 54: 'oak shortbow (u)',
 56: 'oak longbow (u)',
 58: 'willow longbow (u)',
 60: 'willow shortbow (u)',
 62: 'maple longbow (u)',
 64: 'maple shortbow (u)',
 66: 'yew longbow (u)',
 68: 'yew shortbow (u)',
 70: 'magic longbow (u)',
 72: 'magic shortbow (u)',
 91: 'guam potion (unf)',
 93: 'marrentill potion (unf)',
 95: 'tarromin potion (unf)',
 97: 'harralander potion (unf)',
 99: 'ranarr potion (unf)',
 101: 'irit potion (unf)',
 103: 'avantoe potion (unf)',
 105: 'kwuarm potion (unf)',
 107: 'cadantine potion (unf)',
 109: 'dw

Some items are *only* distinguishable via their unique ID number (or of course, by looking at the item in game). If we already knew the unique IDs, we could search `tradeables_ids_to_names` directly for the name. However, I don't have IDs memorized and neither should you.

Next, I'll write a lookup function. If the lookup is "simple" just return a single ID as an integer. If the lookup is "hard", return the last found ID as a single integer, by default. Finally, there will be some parameters for customizing the output.

In [4]:
def dict_to_df(dictionary, cols):
    """
    Turn a Python dictionary into a pandas.DataFrame
    """
    return pd.DataFrame(list(dictionary.items()), columns=cols)

In [5]:
def lookup_id_by_name(name, verbose=True, result=-1):
    name = name.lower()
    df_ids_to_names = dict_to_df(tradeables_ids_to_names, ["ID", "Name"])
    cand = df_ids_to_names[df_ids_to_names["Name"]==name]
    if cand.empty:
        print(f"No matches found for {name}. Returning None")
        return None
    else:
        if verbose:
            display(cand)
        try:
            r = cand.values[result][0]
            return r
        except IndexError:
            print(f"Index {result} out of bounds for found matches. Returning None")
            return None
    #end lookup_id_by_name

API requests follow this basic structure:   
`BASE_URL+ITEM_ENDPOINT`  
Where `ITEM_ENDPOINT` can depend on the item name and id

In [6]:
#API request prefix
BASE_URL = "http://services.runescape.com/m=itemdb_oldschool"
#will return JSON with closing and average price data for last 180 days
PRICE_ENDPOINT = "/api/graph/{}.json"
#will return HTML of the OSRS GE webpage
VOLUME_ENDPOINT = "/{}/viewitem?obj={}"

## GET Price and Average Data ##
Now that I have an easy way of getting item names and ID numbers, let's get the price data (close and average) by populating `PRICE_ENDPOINT` with this information.

In [7]:
def get_price_json(uniqueID):
    """
    Request JSON containing closing and average price of an item
    """
    return requests.get(BASE_URL+PRICE_ENDPOINT.format(uniqueID)).json()

In [8]:
def reindex_by_date(df, date_col, unit="ms"):
    """
    Convert values in dates column to pandas.datetime objects.
    Rename dates column to "Dates"
    Reindex dataframe by "Dates"
    """
    df[date_col] = pd.to_datetime(df[date_col], unit=unit)
    df = df.set_index(date_col)
    df.index.name = "Dates"
    return df

In [9]:
def price_pipeline(name):
    """
    Create a dataframe of price data (close and average) for a specific item.
    """
    #search for the unique ID for the item
    uniqueID = lookup_id_by_name(name)
    #GET price data from API
    series = get_price_json(uniqueID)
    #JSON -> pd.DataFrame
    close = dict_to_df(series["daily"], ["ms", "Close (GP)"])
    #clean up data
    close = reindex_by_date(close, "ms")
    average = dict_to_df(series["average"], ["ms", "Average (GP)"])
    average = reindex_by_date(average, "ms")
    return pd.concat([close, average], axis=1) #horiz. stack close and average series

In [10]:
items_to_lookup = ["Old School Bond", "Chinchompa", "Yew Logs", "Elven top"]
for name in items_to_lookup:
    display(price_pipeline(name))

,ID,Name
2985,13190,old school bond


,Close (GP),Average (GP)
Dates,,
2020-07-17,4561944,4159177
2020-07-18,4490635,4171956
2020-07-19,4386156,4180193
2020-07-20,4344162,4190789
2020-07-21,4521090,4210253
...,...,...
2021-01-08,4697932,5714013
2021-01-09,4541762,5649793
2021-01-10,4626003,5588612


,ID,Name
2269,10033,chinchompa


,Close (GP),Average (GP)
Dates,,
2020-07-17,778,844
2020-07-18,778,838
2020-07-19,778,833
2020-07-20,759,828
2020-07-21,759,822
...,...,...
2021-01-08,859,777
2021-01-09,882,783
2021-01-10,882,788


,ID,Name
567,1515,yew logs


,Close (GP),Average (GP)
Dates,,
2020-07-17,204,238
2020-07-18,199,235
2020-07-19,195,232
2020-07-20,203,229
2020-07-21,210,227
...,...,...
2021-01-08,277,313
2021-01-09,277,312
2021-01-10,275,311


,ID,Name
3627,24009,elven top
3629,24015,elven top
3631,24021,elven top
3633,24027,elven top


,Close (GP),Average (GP)
Dates,,
2020-07-17,6930,6645
2020-07-18,6930,6659
2020-07-19,6930,6674
2020-07-20,6930,6689
2020-07-21,6930,6704
...,...,...
2021-01-08,6985,7242
2021-01-09,6985,7229
2021-01-10,6985,7216


## Basic Tradeables Class ##

Now that I've replicated the basic search function of the Old School Runescape Grand Exchange website, I can add this feature to a tradeables class so I can use it elsewhere.

In [ ]:
class Tradeable:
    
    def __init__()